<a target="_blank" href="https://colab.research.google.com/github/WSU-CS1410-AA/cs1410-notebooks/blob/main/Notebook10-virtual_functions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Multiple inheritance

Unlike other object-oriented programming languages like Java and C#, C++ supports multiple inheritance. That means, in C++, a class can inherit the public and protected members of more than one class. For example, class `C` below inherits from both classes `A` and `B`. This is depicted in the following class hierarchy:

```text
    A                B
    △                △
    |                |
    |                |
    +------- C ------+
```
The triangle points to the parent class.

Here is how these classes are coded.

```cpp
class A { // Base class 1
};

class B { // Base class 2
};

class C : public A, public B { //Derived class from A and B
};
```

## CODING CHALLENGE 1
Define the classes that implement the following hierarchy with Manager and Scientist each inheriting from both Employee and Student.

```text
    Employee                      Student
        △                            △
        |                            |
        |                            |
        +--------- Manager ----------+
        |                            |
        |                            |
        +---------Scientist ---------+
```

In [1]:
%%writefile ch01.cpp

//TODO

Writing ch01.cpp


## Issues with multiple inheritance
Multiple inheritance has its issues, which is why most modern programming languages do not support it. To start, multiple inheritance can lead to overcomplicated program designs. It also suffers from a set of problems, one of which is the infamous **diamond probelm**. Here is an example of such a problem.

```text
        +--------▷ Parent ◁ ---------+    
        |                            |
        |                            |
      Child1                      Child2
        △                            △
        |                            |
        |                            |
        +-------- GrandChild --------+
```    

```cpp
class Parent {
public:
   void foo(){}
};

class Child1 : public Parent {
public:
   void bar(){}
};

class Child2 : public Parent {};
class GrandChild : public Child1, public Child2 {};
```

The problem here is that `Grandchild` has two copies of `foo()`: one inherited from `Parent` via `Child1` and another, also inherited from `Parent`, via `Child2`. So when you call `foo()` on a `GrandChild` object, which one will be used? Well, the compiler will be confused, throwing errors if you try to run something like this:

```cpp
GrandChild d;
d.foo();
```

To the compiler, `d.foo()` is an ambiguous call and it does not know which version of `foo()` to use. It is up to us, the programmers, to clear this up for the compiler and we do that in one of two ways:
* Tell the compiler exactly which version of `foo()` to use using the class name and the resolution operator `::`. For example, the code below, tells the compiler to use the version of `foo()` inherited by `GrandChild` via `Child2`.
  ```cpp
  GrandChild gc;
  gc.Child2::foo();
   ```

* Or use the keyword `virtual` to instruct the compiler to prevent creating multiple copies of functions of the same `Parent`. This means changing the implementation of the above class hierarchy to something like this:

  ```cpp
  class Parent {
  public:
    void foo(){}
  };

  class Child1 : virtual public Parent {
  public:
    void bar(){}
  };

  class Child2 : virtual public Parent {};
  class GrandChild : public Child1, public Child2 {};
  ```

Notice the use of the keyword `virtual` before the public inheritance of the `AnotherChild1` and `AnotherChild2` classes. This tells C++ that if another class inherits from both of these classes (like `AnotherGrandChild` here), only single copies of the functions of the base class `AnotherParent` are made available to it. Thus eliminating the previous ambiguity.

Having done that allows us to run the following without the use of the resolution operator `::`

```cpp
GrandChild gc;
gc.foo();
```

## CODING CHALLENGE 2

Define the classes that implement the following class hierarchy. Use the keyword `virtual` to insure your classes do not suffer from the **diamond problem**.

```text
        +---------▷ BaseClass ◁ ----------+    
        |                                 |
        |                                 |
  DerivedClass1                     DerivedClass2
        △                                 △
        |                                 |
        |                                 |
        +----- DerivedFurtherClass1 ------+
        |                                 |
        |                                 |
        +----- DerivedFurtherClass2 ------+
```

In [2]:
%%writefile ch02.cpp

//TODO

Writing ch02.cpp


# Virtual Functions

In addition to encapsulation and inheritance, **polymorphism** is the third principle of object-oriented programming. It means "multiple forms", and is supported by C++ at both compile-time and run-time. At compile-time, we can think of function overloading and operator overloading as forms of polymorphism, where the same function name or operator is used to mean and do different things in different contexts. At run-time polymorphism is achieved by combining inheritance and **virtual functions**.

Generally speaking, **virtual** means existing in appearance but not in reality. In c++, a **virtual function** is a member function declared at the base class and redefined (or overridden) by a derived class. When this happens, the derived class will have access to two functions with the same name and signature: one provided by the base class and the other by the derived class itself. When a **virtual function** is called, the compiler defers deciding on which function definition to use until run-time. We call this **late binding**, which is different from the **early binding** utilized by non-virtual functions.

To understand **virtual functions** and the difference between **early binding** and **late binding**, let's look at an example. First here is our base class:

In [3]:
%%writefile ex01.cpp

#include <iostream>
using namespace std;

class Base {
public:
    void show() { cout << "Base\n"; }
};

Overwriting ex01.cpp


Next let's create two "regular" classes that derive from it. Both of these classes override the `show()` function. No `virtual` functions yet.

In [4]:
%%writefile -a ex01.cpp

class Derived1 : public Base {
public:
    void show() { cout << "Derived 1\n"; }
};

class Derived2 : public Base {
public:
    void show() { cout << "Derived 2\n"; }
};

Appending to ex01.cpp


Using these classes, we can create the following objects:

In [5]:
%%writefile -a ex01.cpp

int main() {
  Derived1 d1;
  Derived2 d2_1, d2_2;

Appending to ex01.cpp


Now we call the `show()` functions on these objects:

In [6]:
%%writefile -a ex01.cpp

d1.show();
d2_1.show();
d2_2.show();

Appending to ex01.cpp



Let's now see what happens when `Base` pointers are used.

In [7]:
%%writefile -a ex01.cpp

Base* bptr;
bptr = &d1;
bptr->show();
bptr = &d2_1;
bptr->show();

Appending to ex01.cpp


Here the `show()`  function of the `Base` class is called instead of the ones from the derived classes. This  is inspite of this  pointer `bptr` pointing to a `Derived1` object first and a `Derived2` object after that. This is example of **early binding** with the compiler determining which version of `show()` to use at compile-time based on the type of the pointer, which is `Base`.

Let's try the same thing with a `Base` reference:

In [8]:
%%writefile -a ex01.cpp

  Base& bref = d2_2;
  bref.show();

  return 0;
}

Appending to ex01.cpp


Let's compile this program and verify its results:

In [9]:
!g++ -std=c++17 ex01.cpp -o ex01
!./ex01

Derived 1
Derived 2
Derived 2
Base
Base
Base


These calls are behaving as expected, with each invoking the `show()` method defined in its respective class, as determined by the compiler at compile-time.

## Late binding

With **Late binding**, the version of the function is decided based on the actual type of the object at run time. To enable **late binding**, we need to use **virtual functions**. Here is an example of virtual functions starting with the base class (refactored0:

In [10]:
%%writefile ex02.cpp

#include <iostream>
using namespace std;

class Base {
public:
    virtual void show() { cout << "Base\n"; }
    virtual ~Base(){} // Virtual destructor
};

Overwriting ex02.cpp


Notice the use of the keyword `virtual` in front of the `show()` function. This makes this function **virtual** and instructs the compiler to enable **late binding** on it. *This is different from the previous use of virtual, which helped with the diamond problem.*

Notice that the destructor is also marked as virtual. The destructor should be marked virtual on any class with one or more virtual functions. This is to make sure objects of derived classes are destroyed properly. This applies **polymorphism** and **late binding** to object cleanup and makes sure that the correct destructors based on the actual object types are used when objects are deleted.

Let's define two derived classes of this base class:

In [11]:
%%writefile -a ex02.cpp

class Derived1 : public Base {
public:
    void show() { cout << "Derived 1\n"; }
};

class Derived2 : public Base {
public:
    void show() { cout << "Derived 2\n"; }
};

Appending to ex02.cpp



Notice that we did not use the keyword `virtual` inside these derived class; only inside the base. This is because a function needs only be defined as `virtual` once and in the base class. After that, no matter how many times a it is  inherited, it remains virtual.

Let's now see what **virtual functions** do, starting with actual stack objects:

In [12]:
%%writefile -a ex02.cpp

int main() {
  Derived1 d1;
  Derived2 d2_1, d2_2;

Appending to ex02.cpp


First, let's call the `show()` function using the **dot operator**.

In [13]:
%%writefile -a ex02.cpp

d1.show();
d2_1.show();
d2_2.show();

Appending to ex02.cpp


This should work as expected, printing:
```txt
Derived 1
Derived 2
Derived 2
```

Let's see what happens when `Base` pointers are used.

In [14]:
%%writefile -a ex02.cpp

Base* bptr;
bptr = &d1;
bptr->show();

bptr = &d2_1;
bptr->show();

Appending to ex02.cpp


With pointers, the correct `show()` function from the actual class of the object. The first `show()` function was called on a `Derived1` object pointed to by the `bptr` pointer, so `"Derived 1"` is printed. Then the `bptr` pointer is re-pointed to a `Derived2` object. This is why the second `show()` function correctly prints `"Derived 2"`. This is **late binding** in action.

Let's try the same thing, but  with a `Base` reference instead of a pointer:

In [15]:
%%writefile -a ex02.cpp

  Base& bref = d2_2;
  bref.show();

  return 0;
}

Appending to ex02.cpp


Here too, the correct `show()` function of the actual object should be called (The `"Derived 2"` is printed instead of `"Base"`).

Let's compile this program and verify its results:

In [16]:
!g++ -std=c++17 ex02.cpp -o ex02
!./ex02

Derived 1
Derived 2
Derived 2
Derived 1
Derived 2
Derived 2




To summarize, A **virtual function**:
* is a member function that is declared as `virtual' in a base class and redefined (or overridden) by one or more derived classes.
* works like a *normal* non-virtual function when accessed by the **dot operator**.
* supports **late binding** as opposed to **early binding** when accessed via a pointer or a reference.


## CODING CHALLENGE 3(4 parts)

**PART 1**: Define a class named `Rectangle` with four data members:
* two integers x and y for the coordinates of the upper left corner.
* two integers width and height for its width and height.

This class should have a constructor that takes four arguments one for each of its data members. It should also have a virtual function named `getName()` that returns the string `"Rectangle"` and a virtual destructor.


In [17]:
%%writefile ch03.cpp

#include <iostream>
#include <string>

//TODO: The Rectangle class

Writing ch03.cpp


**PART 2**: Knowing that a square is a rectangle whose width and height are the same, define a class names `Square` that inherits from `Rectangle` This class should have a constructor that takes three arguments (x, y, and width) and delegates initialization to the `Rectangle` constructor. It should also override the `getName()` function by returning the string `"Square"`.

In [18]:
%%writefile -a ch03.cpp

//TODO: The Square class

Appending to ch03.cpp


**PART 3**: Create two objects: a rectangle and a square. Also, create a rectangle pointer named `rptr` and initialize it to `nullptr`. Then:

* point the `rptr` pointer to the rectangle object and use it to call the `getName()` function.
* point the `rptr` pointer to the square object and use it to call the `getName()` function.

In [19]:
%%writefile -a ch03.cpp

int main(){

  //TODO: Don't close the main yet.

Appending to ch03.cpp


**PART 4**: Create another two objects: a rectangle and a square. Create a rectangle reference named `rref` and make it an alias to the square object. Use it to call the `getName()` function.

In [20]:
%%writefile -a ch03.cpp

  //TODO

  // Closing the main
  return 0;
}

Appending to ch03.cpp


In [21]:
!g++ -std=c++17 ch03.cpp -o ch03
!./ch03

## Pure functions

Finally, there is the notion of **pure virtual functions**, which are defined by having the `= 0` at the end of their prototypes. A **pure virtual function** is a function that has no actual definition within the class that declares it; only the prototype is provided. This makes any class declaring one or more pure functions **an abstract class**. which is a class whose implementation is not complete and therefore cannot be used to create objects.

Here is an example abstract class.

```cpp
class SomeClass {
public:
    virtual void print() = 0; // a pure function
};
```

Having a **pure virtual function** makes this class **abstract**, which means it is not complete (or concrete) and cannot, therefore, be used to create objects. For example, the following statement throws a compilation error, because `print()` is not implemented, making `SomeClass` an abstract class.

```cpp
SomeClass sc;
```

To summarize, **abstract classes** are meant to be subclassed or inherited by other classes. Their **pure functions** are like holes that make them incomplete and unfit to be used for creating objects.